In [1]:
import os.path
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
pd.set_option('display.max_columns', None)


C:\Users\peppe\AppData\Local\Temp\ipykernel_26196\2666124188.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def read_csv_files(): #reads in data from csv files in sheet
    data_dir = './../data/raw'
    #selecting file paths
    file_paths = [
        os.path.join(data_dir, 'nga_U5_population.xls'),
        os.path.join(data_dir, 'NGA_2018_GHSP-W4_v03_M_CSV', 'sect1_harvestw4.csv'),
        os.path.join(data_dir, 'NGA_2018_GHSP-W4_v03_M_CSV', 'sect2_harvestw4.csv'),
        os.path.join(data_dir, 'NGA_2018_GHSP-W4_v03_M_CSV', 'nga_householdgeovars_y4.csv'),
        os.path.join(data_dir, 'NGA_2018_GHSP-W4_v03_M_CSV', 'sect10b_harvestw4.csv'),
        os.path.join(data_dir, 'NGA_2018_GHSP-W4_v03_M_CSV', 'secta1_harvestw4.csv')
    ]
    
    #reading in indidual csv files containing the survey data
    states_under5_pop = pd.read_excel(file_paths[0])
    key_socio_sect1_harvestw4 = pd.read_csv(file_paths[1])
    key_socio_sect2_harvestw4 = pd.read_csv(file_paths[2])
    nga_householdgeovars_y4 = pd.read_csv(file_paths[3])
    foodspending_sect10b_harvestw4 = pd.read_csv(file_paths[4])
    hhidagland_secta1_harvestw4 = pd.read_csv(file_paths[5])
    #reading the shape files
    households_gdf = gpd.read_file(os.path.join(data_dir, 'NG_202404', 'NG_202404_ML1.shp')) #shp files for spatial joins
    households_gdf2 = gpd.read_file(os.path.join(data_dir, 'NG_202404', 'NG_202404_ML2.shp'))
    
    return states_under5_pop, key_socio_sect1_harvestw4, key_socio_sect2_harvestw4, nga_householdgeovars_y4, foodspending_sect10b_harvestw4, hhidagland_secta1_harvestw4, households_gdf, households_gdf2
states_under5_pop, key_socio_sect1_harvestw4, key_socio_sect2_harvestw4, nga_householdgeovars_y4, foodspending_sect10b_harvestw4, hhidagland_secta1_harvestw4, households_gdf, households_gdf2 = read_csv_files()

C:\Users\peppe\AppData\Local\Temp\ipykernel_26196\429215169.py:16: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  key_socio_sect2_harvestw4 = pd.read_csv(file_paths[2])
C:\Users\peppe\AppData\Local\Temp\ipykernel_26196\429215169.py:18: DtypeWarning: Columns (13,22) have mixed types. Specify dtype option on import or set low_memory=False.
  foodspending_sect10b_harvestw4 = pd.read_csv(file_paths[4])


In [3]:
columns_to_keep = ["ADMIN1", "ADMIN2", "ADMIN3", "LZNAME", "ML1", "geometry"] #column names from boundary data (state data) that is used to merge points within

def spatial_join(remaining_cols, hhid_data, boundary_data): 
    # Filter columns in boundary_data
    boundary_data = boundary_data[remaining_cols]   
    
    # Convert latitude and longitude columns to geometry for merging
    hhid_data['latitude_longitude'] = gpd.points_from_xy(hhid_data['lon_dd_mod'], hhid_data['lat_dd_mod']) 
    hhid_data = gpd.GeoDataFrame(hhid_data)
    hhid_data = hhid_data.set_geometry("latitude_longitude")
    hhid_data.crs = 'EPSG:4326' 
    
    # Convert hhid_data to the same CRS as boundary_data
    hhid_data = hhid_data.to_crs('EPSG:4326')
    
    # Spatial join
    hhid_data = gpd.sjoin(hhid_data, boundary_data, how='inner', op='within') 
    
    # Drop unnecessary columns from the spatial join
    hhid_data = hhid_data.drop(columns=['index_right']) 
    
    # Rename the 'ML1' column to 'at_risk_of_famine' in hhid_data
    if 'ML1' in hhid_data.columns:
        hhid_data.rename(columns={'ML1': 'at_risk_of_famine_current'}, inplace=True)
        hhid_data = hhid_data[['hhid', 'ADMIN1', 'ADMIN2', 'at_risk_of_famine_current']]

    elif 'ML2' in hhid_data.columns:
        hhid_data.rename(columns={'ML2': 'at_risk_of_famine_future'}, inplace=True)
        hhid_data = hhid_data[['hhid', 'ADMIN1', 'ADMIN2', 'at_risk_of_famine_future']]

   
    
    return hhid_data


master = spatial_join(columns_to_keep, nga_householdgeovars_y4, households_gdf)



C:\Users\peppe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3553: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
#appending future at risk of famine column
columns = ["ADMIN1", "ADMIN2", "ADMIN3", "LZNAME", "ML2", "geometry"]
master2 = spatial_join(columns,nga_householdgeovars_y4,households_gdf2)
master = master.merge(master2, on='hhid', how='left')


C:\Users\peppe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3553: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
master.columns

Index(['hhid', 'ADMIN1_x', 'ADMIN2_x', 'at_risk_of_famine_current', 'ADMIN1_y',
       'ADMIN2_y', 'at_risk_of_famine_future'],
      dtype='object')

In [6]:
master.drop(columns=['ADMIN1_x','ADMIN2_x'])

,hhid,at_risk_of_famine_current,ADMIN1_y,ADMIN2_y,at_risk_of_famine_future
0,10001,1,Abia,None,2
1,10002,1,Abia,None,2
2,10003,1,Abia,None,2
3,10004,1,Abia,None,2
4,10005,1,Abia,None,2
...,...,...,...,...,...
5102,379148,2,Federal Capital Territory,None,2
5103,379151,2,Federal Capital Territory,None,2
5104,379152,2,Federal Capital Territory,None,2
5105,379154,2,Federal Capital Territory,None,2


In [7]:
#Merging socioeconomic datat
def socio_merge(hhid_data):
    
    hhid_data = pd.merge(key_socio_sect1_harvestw4, hhid_data, on='hhid', how='inner')
    #Adding questions from socio survey
    hhid_data = hhid_data[['hhid', 'ADMIN1_y', 'ADMIN2_y', 'at_risk_of_famine_current','at_risk_of_famine_future','indiv','s1q2','s1q4a','s1q4','s1q7','s1q11','s1q17','s1q18','s1q20','s1q21','s1q22','s1q23','s1q30']]
    hhid_data.rename(columns={'s1q2': 'sex', 's1q4a': 'member_of_household','s1q4':'age','s1q7':'marital_status','s1q11' : 'partner_in_household','s1q17': 'religion','s1q18': 'father_in_household', 's1q20': 'father_alive' ,'s1q21':'highest_father_education','s1q22':'father_industry','s1q23':'mother_in_household','s1q30':'in_nigeria'}, inplace=True)
    hhid_data['household_size'] = master.groupby('hhid')['hhid'].transform('count')
    merged_soc = pd.merge(key_socio_sect2_harvestw4[['hhid', 's2aq3', 's2aq5', 's2aq6','indiv']], hhid_data, on=['hhid', 'indiv'], how='inner')
    merged_soc = merged_soc.rename(columns={'s2aq3': 'answering_for_self', 's2aq5': 'read_or_write', 's2aq6': 'attended_school'})
    return merged_soc
socio_df = socio_merge(master)


In [8]:
#Merging food spending data
def food_df(social_data):
    #Creating new column for foodspending by household in last 30 days
    sum_by_hhid = foodspending_sect10b_harvestw4.groupby('hhid')['s10bq10'].sum()
    sum_by_hhid_df = sum_by_hhid.reset_index(name='total_spent_on_food_in_last_30')
    merged_food = pd.merge(social_data, sum_by_hhid_df, on='hhid', how='inner')
    return merged_food
merged_food_df = food_df(socio_df)

In [9]:
def under_5_pop_df(merged_food):
    #column that shows the under 5 population in the state the survey respondent lives in
    merged_food = merged_food.rename(columns={'ADMIN1_y': 'State'})
    merged_food = pd.merge(merged_food, states_under5_pop, on='State', how='inner')
    return merged_food
under_5_df = under_5_pop_df(merged_food_df)

In [10]:
#adding farming related questions
def land_df(population_df):
    population_df = pd.merge(population_df, hhidagland_secta1_harvestw4[['hhid', 'zone', 'state', 'lga', 'sector', 'ea', 'sa1q1aa', 'prefilled_gps_area', 'sa1q6', 'sa1q11']], on='hhid', how='inner')
    population_df = population_df.drop_duplicates(subset=['hhid', 'indiv'])
    master = population_df.rename(columns={'sa1q1aa': 'cultiavted_plot', 'sa1q6': 'plot_cost', 'sa1q11': 'plot_area','ADMIN2': 'city'})

    return master
master = land_df(under_5_df)

In [11]:
#convert dataset to excel workbook
master.to_excel('master_data.xlsx', index=False)


: 